<a href="https://colab.research.google.com/github/davidberger2785/MATH80600/blob/main/Introduction_%C3%A0_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semaine 4: Introduction à Pytorch


**Auteurs**:

David Berger (davidberger2785 [at] gmail [dot] com)

Le présent atelier s'est (grandement) inspiré des ateliers de: 
1. Mang Qu et Zhaocheng Zhu dans le cadre du cours MATH 80600A à l'hiver 2021 à HEC
2. Chin-Wei Huang dans le cadre du cours IFT 6135 à l'hiver 2019 à l'Université de Montréal.


# 1. Introduction


Dans cet atelier, nous allons apprendre comment utliser la bibliothèque Pytorch et, accessoirement, comment implémenter des réseaux de neurones à l'aide cette dernière. 

L'ensemble du code ci-bas peut être exécuté directement sur Google Colab. Utiliser cette plate-forme présente plusieurs avantages, le principal étant de proposer aux usagers.ères l'accès à un GPU, ce qui accélère grandement le temps de calcul pour des tâches plus lourdes.


Avant d'exécuter l'ensemble des commandes de ce tutoriel, assurez-vous que d'utiliser un GPU. Pour ce faire:
1. Sélectionnez *Exécution &gt; Modifier le type d'exécution*.
2. Dans le menu déroulant *Accélérateur matériel*, sélectionnez l'option *GPU*.

# 2. Installation des bibliothèques

Installons les bibliothèques d'usage en [langage Python](https://www.youtube.com/watch?v=RpOzFBSwSLc).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from __future__ import print_function

Enfin, nous installons quelques bibliothèques propres à Pytorch.

In [ ]:
!pip install -q torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision

Vérifier qu'un GPU est bien disponible pour le notebook peut s'avérer une bonne idée. Pour ce tutoriel, la sortie associée devrait est True.

Bien qu'il soit possible d'avoir accès à un GPU avec Colab, notez que toute commande, ou opération, n'implique pas nécessairement l'utilisation d'un GPU.

In [ ]:
torch.cuda.is_available()  # Afin de vérifier si un GPU est disponible

# 3.0 Rudiments sur les tenseurs

En apprentissage profond, les données sont représentées à l'aide de [tenseurs](https://fr.wikipedia.org/wiki/Tenseur) que l'on pourrait grossièrement considérer comme une généralisation des matrices. Dans cette section, nous réviserons quelques commandes fondamentales sur les tenseurs en Pytorch.

## 3.1 Une image de chat

Téléchargeons dans premier temps une image et réprensatons-la sous la forme d'un tenseur à l'aide de Pytorch.

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/b/b6/Felis_catus-cat_on_snow.jpg -O cat_winter.jpg

In [ ]:
from PIL import Image

np_image = np.array(Image.open("cat_winter.jpg"))
image = torch.as_tensor(np_image)
plt.imshow(image)

Par convention:
- La **dimension** refère à l'un des axes du tenseur.
- La **taille** refère à la longeur d'un axe choisi pour une dimension choisie d'un tenseur.
- L'**index** refère à une coordonnée particulière du tenseur.

À titre d'exemple, l'image ci-dessus présente trois dimensions. Les deux premières dimensions ont des tailles de 2000 et 3000 respectivement.

In [ ]:
print("Nombre de dimensions:", image.ndim)
print("Taille de chaque dimension:", image.shape)

**Questions 3.1**

1. À quoi est associée à la troisième dimension du tenseur dans l'image ci-haut?

2. Trouvez une façon de visualiser l'une de ces dimensions.

**Réponses 3.1**

Aux trois canaux RGB. On peut visualiser l'un des trois canaux en fixant simplement l'un des indices.

In [ ]:
plt.imshow(image[:, :, 0]) 


**Opérations arithmétiques**

Les opérations sur les tenseurs sont analogues à celles rencontrées dans un cours d'introduction en algèbre linéaire où l'accent est mis sur les opérations matricielles. Par exemple, pour chacun des pixels de l'image téléchargée, nous pouvons calculer la valeur moyenne de la troisième dimension.

In [ ]:
channel_mean = image.float().mean(dim=2)
print(channel_mean.shape)

La moyenne calculée, nous pouvons visualiser la nouvelle image, où toute couleur devrait être évacuée.

In [ ]:
plt.imshow(channel_mean,  cmap="gray")

**Coordonnées**

Nous pouvons également couper l'image en quatre parties disjointes.

In [ ]:
y_dim, x_dim = int(image.shape[0] / 2), int(image.shape[1] / 2)

crop_up_left = image[:y_dim, :x_dim, :]
crop_up_right = image[:y_dim, x_dim:, :]
crop_bottom_left = image[y_dim:, :x_dim, :]
crop_bottom_right = image[ y_dim:, x_dim:, :]

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(crop_up_left)
axarr[1,0].imshow(crop_bottom_left)
axarr[0,1].imshow(crop_up_right)
axarr[1,1].imshow(crop_bottom_right)

**Transposition**

Nous pouvons transposer les deux premiers axes de l'image.

In [ ]:
transposition = image.transpose(0, 1)   # Transposition du premier (0) et du deuxième (1) axes
plt.imshow(transposition)

**Changement de dimensions**

Nous pouvons réduire les dimensions d'un tenseur. Ce genre de manipulation est typique si nous voulons effectuer une opération sur plusieurs axes de façon simultannée comme nous le ferons à la Section 7.


In [ ]:
flat = image.flatten(0, 1)
print(flat.shape)
plt.imshow(flat[:10, :])

## 3.2 Délaissons les chats

Délaissons les images de chats et travaillons sur des objets plus abstraits. Dans cette section, nous présenterons en rafale quelques opérations utiles sur les tenseurs qu'il est possible d'effectuer en Pytorch.

###3.2.1 Initialisation d'un tenseur aléatoire

In [ ]:
torch.Tensor(5, 3)

Pour initialisation à partir d'une distribution choisie, on peut se référer à [cette page](https://pytorch.org/docs/stable/torch.html#in-place-random-sampling) sur le site de Pytorch.

In [ ]:
mu, sigma = 0, 1
torch.Tensor(5, 3).normal_(mu, sigma)

Ou encore en effectuant cette commande:

In [ ]:
torch.normal(mu, sigma, size=(5, 3))

###3.2.2 Construction de tenseurs particuliers

Il est possible de construire des tenseurs uniquement constitués de 0 ou de 1.

In [ ]:
longueur = 5
torch.ones(longueur), torch.zeros(longueur)

###3.2.3 Type d'un tenseur

Dans certains cas, le type de tenseur est important. Plus de détails sur les types [ici](https://pytorch.org/docs/master/tensors.html)!


In [ ]:
z = torch.Tensor([[1, 3], [2, 9]])
print(z.type())
print(z.numpy().dtype)

z_ = torch.LongTensor([[1, 3], [2, 9]])
print(z_.type())
print(z_.numpy().dtype)

###3.2.4 Manipulations arithmétiques

Plusieurs opérations arithmétiques sont possibles.

In [ ]:
x = torch.Tensor(2, 2).uniform_(0, 1)
y = x ** torch.eye(2)
print(y)

Et surprenemment simple à exécuter.

In [ ]:
noise = torch.randn(2, 2)
y = x / torch.sqrt(noise ** 2)
print(y)

###3.2.5 Diffusion

Nous pouvons par exemple additionner à la première dimension dimension d'un tenseur $\mathbf{x}$ un tenseur $\mathbf{y}$ dont les dimensions sont identiques aux dimensions subséquentes de $\mathbf{x}$. 

In [ ]:
print(x)
y = x + torch.arange(2)
print(y)
# print(x + torch.arange(5))

###3.2.6 Manipulation des dimensions

Apprendre à manipuler convenablement les dimensions des tenseurs est un art perdu. Franchement, le meilleur truc est de pratiquer, et de se souvenir que pareilles manipulations existent!

In [ ]:
y = torch.randn(2, 3, 4)
print("Taille originale: ", y.size())
print("Mixer les deux premières dimension: ", y.view(-1, 4).size()) 

indice = 1
print("Ajout de dimension à l'indice: ", y.view(-1, 4).unsqueeze(indice).size()) 
print(y.view(-1, 4).unsqueeze(1).unsqueeze(2).unsqueeze(3).squeeze().size()) # cette commande est franchement dégueulasse

Et apprendre à transposer est une bonne idée également!

In [ ]:
print(y.transpose(0, 1).size())
print(y.transpose(1, 2).size())
print(y.transpose(0, 1).transpose(1, 2).size())
print(y.permute(1, 2, 0).size())

###3.2.7 Concaténation de tenseurs

In [ ]:
print("Dimension originale:", y.shape)

k = 2 # dimension dans laquelle nous voulons concatener
print("Concaténation sur la kième dimension:", torch.cat([y, y], dim=k).size())

# stack empile les tenseurs dans une nouvelle dimension
print("Empilons les tenseurs avec stack:", torch.stack([y, y], 0).size())

###3.2.8 Indexation avancée

In [ ]:
rev_idx = torch.arange(1, -1, -1).long()
print(rev_idx)
print(y[rev_idx].size())   # pour inverser les tenseurs de la premiere dimension

Enfin, [torch.gather](https://pytorch.org/docs/1.9.1/generated/torch.gather.html) peut être une fonctionnalité utile.

In [ ]:
v = torch.arange(12).view(3,4)   # equivalent à reshape
print(v)
print(v.shape)

# Supposons que l'on veut retourner les element [1], [6] et [8]
print(torch.gather(v, 1, torch.tensor([1,2,0]).long().unsqueeze(1)))


###3.2.9 Navigueur des CPUs aux GPUs (et vice versa)

In [ ]:
x = torch.FloatTensor(5, 3).uniform_(-1, 1)
print(x)
x = x.cuda()
print(x)
x = x.cpu()
print(x)

##3.3 Remarque rapide sur les fonctions en Pytorch

Enfin, il y a vraiment pleins d'opérations plus sophistiquées qui ont été implémentées par le passé. La règle du pouce: si vous êtes capable d'y donner un nom, elle existe en Pytorch! Si vous comprenez bien ce que la fonction fait, cela ne vaut pas la peine de l'implémenter à la mitaine. Surtout que la plupart des fonctions sont robustes aux problèmes d'instabilité numérique...

Par exemple, supposons que $𝐰$ est un tenseur de taille $d$. La fonction $\text{softmax}(𝐰)$ est définie ainsi:


$$\\ \text{softmax}(𝐰)_i := \frac{\exp(𝐰_i)}{\sum_{k=1}^d \exp(𝐰_k)},$$

où $\exp (\cdot)$ est la fonction exponentielle.

In [ ]:

w = torch.tensor([1., 2., 3., 4., 5.])
w_cuda = w.cuda()

w_exp = w.exp()
w_sum = w_exp.sum()
w_exp/w_sum

Naturellement, la fonction softmax est directement [implémentée en Pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)!

In [ ]:
torch.softmax(w, dim=0)

# 4.0 Calcul du gradient et différentiation automatique

Estimer les paramètres régissant les réseaux de neurones est souvent une tâche complexe. La fonction de perte à optimiser n'admettant pas une solution analytique, la descente de gradient s'avère une technique indispensable pour estimer les paramètres de pareilles architectures. Or, avant l'arrivée des bibliothèques d'apprentissage profond, chercheur.e.s et praticien.ne.s se voyaient obligés d'expliciter l'expression du gradient pour chaque paramètre du réseau. Si cet exercice est plus ou moins pénible pour des MLPs (amusez-vous à coder un MLPs seulement en Numpy!), il se complique lorsque les architectures se complexifient. Pensez à un ResNet ou encore à un LSTM.

En ce sens, le calcul du gradient par différentiation automatique est un outil précieux qui vous permettra, avec un peu de pratique, d'implémenter et de développer des architectures complexes. Par contre, il est nécessaire de bien saisir comment la différentiation automatique procède. 

La présente section vise à introduire la différentation automatique selon Pytorch. Elle peut paraître rébarbative à prime à bord, mais les bénéfices à moyens et longs termes en valent la peine.

Enfin, le matériel présenté ici s'inspire principalement de deux sources à savoir [le tutoriel d'introduction à Autograd par Pytorch](http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py) et [une introduction à la mécanime d'Autograd par Pytorch](http://pytorch.org/docs/master/notes/autograd.html).

## 4.1 Ce qu'il faut savoir

Les réseaux de neurones peuvent être considérés comme une succession de fonctions, lesquelles sont, ou pas, embriquées les unes dans les autres. Chaque fonction est dès lors définie grâce à un ensemble de paramètres dont les valeurs sont mémorisées à l'aide de tenseurs.

Comme nous l'avons présenté lors du deuxième cours, l'entraînement d'un réseau de neurones peut être défini en deux étapes distinctes:

1. **Propagation du message vers l'avant**: où, en fonction du modèle estimé, une prédiction d'une issue est faite en fonction d'un ensemble d'entrée.

2. **Rétro propagation vers l'arrière**: où, en fonction de l'issue estimée lors de la première étape et de la précision de cette mesure par rapport à sa valeur réelle, une correction des paramètres du modèle est effectuée.

Dans cette section, nous présentons en quelques étapes simples comment utiliser la différentiation automatique pour entraîner un réseau de neurone avec Pytorch. Cette section n'introduit aucunement la mécanique derrière chacune des fonctions utilisées. 

### 4.1.1 Initialisation du modèle

Lors de la séances portant sur les CNNs, nous avons brièvement introduit les ResNets. Nous proposons donc de télécharger le modèle ResNet18 à partir de torch.vision. Une technique d'optimisation est également choisie, comme par exemple la descente stochastique de gradient, avec un pas d'apprentissage de 0,1. Enfin, on opte pour une fonction de perte basée sur l'erreur quadratique moyenne (MSE).

In [ ]:
model = torchvision.models.resnet18(pretrained=True)   # importation de ResNet18
optimizer = torch.optim.Adam(model.parameters(), lr=.1e1) 
loss = nn.MSELoss()

Aux fins de l'exercices, nous créons un tenseur initialisé de façon aléatoire et l'issue, ou étiquette, qui lui est associée.

In [ ]:
data = torch.rand(1, 3, 64, 64)   # generation d'un tenseur aleatoire
labels = torch.rand(1, 1000)   # generation d'une issue

### 4.1.2 Propagation vers l'avant

Par après, nous propageons l'information du message vers l'avant tel que décrit brièvement au point 1.

In [ ]:
prediction = model(data)    # propagation du message vers l'avant

### 4.1.3 Calcul de la fonction de perte 

Une fois la prédiction calculée, nous calculons l'erreur de prédiction en fonction des vraies valeurs (et de la fonction de perte choisie).

In [ ]:
output = loss(labels, prediction)

### 4.1.4 Calcul du gradient via rétro propagation

L'erreur est alors rétro propagée à travers le ResNet18. Pour chaque paramètre du modèle, un gradient est calculé et nous pouvons obtenir ces valeurs avec la commande suivante.

In [ ]:
output.backward()

### 4.1.5 Mise à jour des paramètres 

Enfin, la descente de gradient peut être effectuée!

In [ ]:
optimizer.step()

Voilà! Vous savez maintenant comment utiliser Autograd pour entraîner un réseau de neurones!

## 4.2 Ce qui vous sera (certainement) utile

Dans cette section, nous proposons de comprendre davantage en profondeur le fonctionnement d'Autograd afin d'effectuer la différentiation automatique. Pour développer notre intuition, nous délaisserons les réseaux de neurones et travaillerons plutôt avec un ensemble restreint de fonctions.

**Initialisation et attribut**: Dans un premier temps, nous initialisons deux vecteurs de paramètres, soit $\mathbf{a}$ et $\mathbf{b}$, pour lesquels nous voulons calculer leur gradient.

In [ ]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

**Fonction simple**: Nous définissons une variable $\mathbf{c}$ laquelle est définie en fonction des vecteurs $\mathbf{a}$ et $\mathbf{b}$ ainsi:

$$ \mathbf{c} = 3 \mathbf{a}^2 - \mathbf{b}$$.

In [ ]:
c = 3 * a**2 - b

**Calcul du gradient de façon analytique**: De façon similaire à ce que nous avons présenté à la section précédente, supposons que $\mathbf{c}$ est l'erreur. Afin d'estimer les paramètres du réseau de neurones, soit $\mathbf{a}$ et $\mathbf{b}$, nous devons calculer les dérivées partielles de $\mathbf{c}$ en fonction des paramètres:

$$ \frac{d \mathbf{c}}{d \mathbf{a}} = 6 \mathbf{a}, \qquad \frac{d \mathbf{c}}{d \mathbf{b}} = -1.$$

**Calcul du gradient avec Autograd**: Nous pouvons calculer les expressions ci-haut simplement à l'aide de la commande .backward().


In [ ]:
c.sum().backward()

Et c'est heureux que les valeurs obtenues concordent avec celles dérivées formellement:

In [ ]:
print(6 * a == a.grad)
print(-1 == b.grad)

**Question**

Quelles valeurs aurions-nous obtenues si nous avions décidé, lors de l'initialisation du vecteur $\mathbf{a}$, de ne pas calculer le gradient pour ce vecteur de paramètres?

#5.0 Classifieur linéaire

Dans cette section, nous allons entraîner un classifieur linéaire plutôt naïf sur le célèbre jeu de données [Iris](https://fr.wikipedia.org/wiki/Iris_de_Fisher). L'idée n'est pas d'utiliser des modèles sophistiqués, mais de simplement se faire tranquillement la main sur les concepts de différentiation automtique vus jusqu'à présent.

**Importatation des bibliothèques**

In [ ]:
from sklearn import datasets
from sklearn.model_selection import KFold

**Préparation des données**

In [ ]:
# importation des donnes
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target

# creation des ensemble d'entrainement et de test
# toujours une bonne idee de brasser la soupe avant de commencer #shuffle
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(y)

for train_index, test_index in kf.split(y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

# transformation des donnees sous la forme de tenseur
X_train, X_test = torch.Tensor(X_train), torch.Tensor(X_test)
y_train, y_test = torch.Tensor(y_train), torch.Tensor(y_test)

**Un modèle**

Dans ce cas-ci, nous allons supposer que les issues suivent le modèle suivant:

$$ y = \beta_0 x_0 + \beta_1 x_1 + \epsilon,$$
où $\epsilon \sim N(0, 1)$. Évidemment, cette modélisation n'est pas appropriée (comprenez-vous pourquoi?), mais elle fera le boulot pour la suite de l'exemple. 

Nous pouvons dès lors initialiser notre modèle avec une classe plutôt simple.Enfin, comprendre comment créer une classe en Python est fondamental pour créer des modèles plus complexes. Assurez-vous de bien comprendre ce concept.

In [ ]:
class model():
  def __init__(self):
    self.params = torch.tensor([[0.], [0.]], requires_grad=True)
  
  def prediction(self, X):
    return(torch.mm(X, self.params).squeeze_())

**Questions**
1. Que pensez-vous de l'initialisation des paramètres du modèle. Est-ce un problème dans ce cas-ci? 
2.Pouvez-vous imaginer un cas d'espèce où ce genre d'initalisation ne serait pas souhaitable?

**Initialisation**

Nous initialisons par la suite le modèle et définissons les hyper paramètres.

In [ ]:
cl = model()   # initialisation du modele
mse = nn.MSELoss()   # fonction de perte choisie

# Quelques hyper paramètres
lr = 1e-2   # pas d'apprentissage
nb_epoch = int(1e2)   # nombre d'epoques

**Apprentissage**

La phase d'entraînement se fait de façon analogue à ce que nous avons vu à la Section.

In [ ]:
mse_train, mse_test = [], []

# Apprentissage
for epoch in range(nb_epoch):
  
  # Prediction (voir le module mm dans Pytorch)
  y_hat = cl.prediction(X_train)

  # Calcul de la fonction de perte (EQM dans ce cas-ci)
  loss = mse(y_hat, y_train)
  mse_train.append(loss)   # pratique pour un graphe futur
 
  # Calcul du gradient
  loss.backward()

  # Mise a jour des parametres du modele
  cl.params.data = cl.params.data - lr * cl.params.grad
  cl.params.grad.zero_()

  mse_test.append(mse(y_test, cl.prediction(X_test)))   # pour un graphe futur

Enfin, nous pouvons étudier rapidement comment le modèle se comporte au fil des itérations à l'aide d'un petit graphique.

In [ ]:
plt.plot(mse_train, label= 'Entraînement')
plt.plot(mse_test, label= 'Test')

plt.xlabel('Epoques')
plt.ylabel('MSE')
plt.legend()

**Questions**
1. Est-ce que les courbes ci-haut se comportent de façon attendue?
2. Comment pourrions-nous fixer ce "problème"?

#6.0 MLP en Pytorch et Skorch

Dans cette section, nous proposons d'implémenter et entraîner un MLP sur le jeu de donnée [MNIST](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST). Ce jeu de données est fameux et consiste à classer des images des chifres manuscrits.

**Téléchargement des données**

In [ ]:
train = torchvision.datasets.MNIST("./data", train=True, download=True)
test = torchvision.datasets.MNIST("./data", train=False, download=True)


**Visualisation des données**

Afin d'avoir une petite idée du jeu de données en question, nous pouvons visualiser quelques images en entrée à l'aide de notre fonction maison plot_mnist.


In [ ]:
def plot_mnist(data, labels=None, num_sample=5):
  n = min(len(data), num_sample)
  for i in range(n):
    plt.subplot(1, n, i+1)
    plt.imshow(data[i], cmap="gray")
    plt.xticks([])
    plt.yticks([])
    if labels is not None:
      plt.title(labels[i])

In [ ]:
train.labels = [train.classes[target] for target in train.targets]
plot_mnist(train.data, train.labels)

**Construction de la classe MLP**

Nous pouvons construire le MLP à l'aide de la classe suivante. Si vous êtes un peu confus.es, ne vous inquiétez pas; ce genre de manipulation deviendra rapidement une seconde nature avec un peu d'entraînement.

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.5):
    super(MLP, self).__init__()
    
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, images):
    x = images.flatten(1)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.softmax(self.fc2(x), dim=-1)
    
    return x

La classe construite, nous pouvons initialiser notre modèle.

In [ ]:
mlp = MLP(
    input_dim=train.data.shape[1] * train.data.shape[2],
    hidden_dim=128,
    output_dim=len(train.classes))

**Skorch**

Dans ce tutoriel, l'entraînement des modèles va se faire à l'aide de [Skorch](https://skorch.readthedocs.io/en/stable/index.html). Cette bibliothèque permet entres autres d'utiliser des modèles implémentés en Pytorch et d'effectuer l'entraînement à l'aide de [Scikit-learn](https://scikit-learn.org/stable/). La phase d'entraînement du MLP diffère donc beaucoup de celle que nous avons présenté à la section précédente, où la mécanique d'Autograd était beaucoup plus explicite. Même si l'interfarce de Skorch est beaucoup plus convivial, je vous encore fortement à comprendre et saisir les nuances présentées à la Section 6.

In [ ]:
pip install -U skorch

In [ ]:
import skorch

Nous faisons la migration du MLP en Skortch et fixons les hyper paramètres associés.

In [ ]:
# Hyper paramètres
num_epoch = 20
lr = 1e-2

# Migration en skorch
model = skorch.NeuralNetClassifier(mlp, max_epochs=num_epoch, lr=lr, device="cuda")

**Entraînement à l'aide de Skorch**

La phase d'entraînement avec Skorch présente automatique les performances de prédiction pour chacune des époques. Une fonction de perte (*loss*) plus faible est préférable et une capacité prédictive (*acc*) plus grande est souhaitable.

In [ ]:
training = model.fit(train.data / 255.0, train.targets)

Nous pouvons calculer l'erreur de prédiction sur l'ensemble de test.

In [ ]:
from sklearn.metrics import accuracy_score

test.mlp_predictions = model.predict(test.data / 255.0)
sklearn.metrics.accuracy_score(test.targets, test.mlp_predictions)

Bon, ce n'est pas fameux... C'est possible d'obtenir de bonnes performances à l'aide d'un MLP. Par contre, cela demande un peu de travail.




**Questions**

1. Modifier la classe du MLP pour augmenter sa capacité, changer les fonctions d'activations, bref, tout ce qui vous passe par la tête afin d'améliorer les capacités prédictives du modèle.

2. Opter pour [une autre technique](https://pytorch.org/docs/stable/optim.html#algorithms) afin d'estimer le gradient peut s'avérer, entre autres stratégies, une bonne idée. Voici un exemple ci-bas montrant comment procéder. Familiarisez-vous avec pareille manipulation et essayez d'obtenir une capacité prédicitve de plus de 96% sur l'ensemble de test.

In [ ]:
import torch.optim as optim   # cette bibliothque propose une pletarde d'optimiseurs

# initalisation du modele
mlp = MLP(
    input_dim=train.data.shape[1] * train.data.shape[2],
    hidden_dim=128,
    output_dim=len(train.classes))

# migration sur Skorch mais avec un nouvel optimiseur
model = skorch.NeuralNetClassifier(mlp, optimizer=optim.RMSprop, max_epochs=10,
                                   lr=1e-4, device="cuda")

# entrainement
model.fit(train.data / 255.0, train.targets)

# predictions sur l'ensemble de test
test.mlp_predictions = model.predict(test.data / 255.0)
test_acc = sklearn.metrics.accuracy_score(test.targets, test.mlp_predictions)

print("\n Peformances sur l'ensemble de test: ", test_acc)

**Sauvegarde du modèle**

Enfin, c'est une bonne idée de sauvegarder notre modèle à la suite de l'entraînement.


In [ ]:
import pickle

with open("MLP.pkl", "wb") as fout:
  pickle.dump(model, fout)
with open("MLP.pkl", "rb") as fin:
  model = pickle.load(fin)

# 7.0 CNN en Pytorch et Skorch

Souvent, utiliser des modèles déjà implémentés peut s'avérer une très bonne idée. Cela sauve beaucoup de temps, entre autres choses... Heureusement, il existe plusieurs modèles classiques de CNN déjà implémentés dans la bibliothèque `torchvision`.

Dans cette section, nous allons utiliser le modèle ResNet-18 (!). Puisque ce modèle a été implémenté pour classer mille types d'images, nous allons simplement réécrire la dernière couche cachée pour l'adapter au jeu de données MNIST.

In [ ]:
resnet18 = torchvision.models.resnet18()
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, len(train.classes))   # overide de la sortie pour l'adapter à MNIST

Encore une fois, puisque ResNet a été implémenté pour traiter des images colorées, nous allons réécrire les images MNIST sous la forme d'un tenseur comportant trois canaux.

In [ ]:
train.color_data = train.data.unsqueeze(1).expand(-1, 3, -1, -1)
test.color_data = test.data.unsqueeze(1).expand(-1, 3, -1, -1)

Comme à la section précédente, nous initialisons notre modèle et l'entrainons.

In [ ]:
# hyper parametres
num_epoch = 2
lr = 1e-1

# initialisation
resnet = skorch.NeuralNetClassifier(
    resnet18, criterion=torch.nn.CrossEntropyLoss, max_epochs=num_epoch, lr=lr,
    device="cuda")

# entrainement
training = resnet.fit(train.color_data / 255.0, train.targets)

Sans surprise, et avec très peu d'effort, ResNet écrase notre MLP un peu vanille...

**Modèle pré entraînées** 

La bibliothèque `torchvision` [propose des modèles pré entraînés](https://pytorch.org/docs/stable/torchvision/models.html) sur des jeux de données distincts. Par exemple, dans ce cas-ci, le ResNet que nous allons téléchargé a été entraîné sur ImageNet. Initialiser notre modèle avec ce genre de paramètres peut s'avérer une bonne stratégie.


In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, len(train.classes))

In [ ]:
# migration vers Skorch
resnet = skorch.NeuralNetClassifier(
    resnet18, criterion=torch.nn.CrossEntropyLoss, max_epochs=num_epoch, lr=lr,
    device="cuda")

# entrainement
training = resnet.fit(train.color_data / 255.0, train.targets)